In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

init_seed = 0
backdata = {}
dataset = ['cifar10', 'cifar100', 'TinyImagenet', 'TinyImagenet']
architecture = ['resnet18', 'resnet50', 'resnet18', 'vgg16_bn']

for arch, data in zip(architecture, dataset):
    for us in range(3):
        with open(f"assets/figures/conf_diff/backdata/{data}_{arch}_unlearn_seed_{us}_confidence_dict.pkl", "rb") as f:
            seed_dict = pickle.load(f)
            backdata[(arch, data, us)] = seed_dict



In [ ]:
dataset = ['cifar10', 'cifar100', 'TinyImagenet', 'TinyImagenet']
architecture = ['resnet18', 'resnet50', 'resnet18', 'vgg16_bn']

for arch, data in zip(architecture, dataset):
    for us in range(3):
        backdata[(arch, data, us)][init_seed]['orig_confidence']['forget'].numpy() # Original-Forget
        backdata[(arch, data, us)][init_seed]['orig_confidence']['retain'].numpy() # Original-Retain
        backdata[(arch, data, us)][init_seed]['orig_confidence']['test'].numpy() # Original-Test

        backdata[(arch, data, us)][init_seed]['retrain_confidence']['forget'].numpy() # Retrain-Forget
        backdata[(arch, data, us)][init_seed]['retrain_confidence']['retain'].numpy() # Retrain-Retain
        backdata[(arch, data, us)][init_seed]['retrain_confidence']['test'].numpy() # Retrain-Test


In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

for i, title in enumerate(["orig", "retrain"]):

    architecture = ['resnet18', 'resnet50', 'resnet18', 'vgg16_bn']
    dataset = ['cifar10', 'cifar100', 'TinyImagenet', 'TinyImagenet']
    unlearn_seeds = range(3)  # 0,1,2
    init_seed = 0  


    dfs_forget = []
    dfs_retain = []
    dfs_test = []

    for arch, data in zip(architecture, dataset):
        # for us in unlearn_seeds:
        us = 0
        beauty_arch = {'resnet18':'R18', "resnet50":'R50', "vgg16_bn":'V16'}
        beauty_dataset = {'cifar10':'CIFAR10', 'cifar100':'CIFAR100', 'TinyImagenet':'TinyImageNet'}
        group_label = f"{beauty_arch[arch]} - {beauty_dataset[data]}"# - unlearn{us}"

        if i == 0:
            forget_values = backdata[(arch, data, us)][init_seed]['orig_confidence']['forget'].numpy()
            retain_values = backdata[(arch, data, us)][init_seed]['orig_confidence']['retain'].numpy()
            test_values = backdata[(arch, data, us)][init_seed]['orig_confidence']['test'].numpy()
        else:
            forget_values = backdata[(arch, data, us)][init_seed]['retrain_confidence']['forget'].numpy()
            retain_values = backdata[(arch, data, us)][init_seed]['retrain_confidence']['retain'].numpy()
            test_values = backdata[(arch, data, us)][init_seed]['retrain_confidence']['test'].numpy()

        # forget
        df_forget = pd.DataFrame({
            "x": forget_values,
            "g": group_label,
        })
        dfs_forget.append(df_forget)

        # retain
        df_retain = pd.DataFrame({
            "x": retain_values,
            "g": group_label,
        })
        dfs_retain.append(df_retain)

        # test
        df_test = pd.DataFrame({
            "x": test_values,
            "g": group_label,
        })
        dfs_test.append(df_test)

    df1 = pd.concat(dfs_retain, ignore_index=True) 
    df2 = pd.concat(dfs_forget, ignore_index=True)
    df3 = pd.concat(dfs_test, ignore_index=True)



    NUM_GROUPS = 4 #12
    # Set theme
    sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

    # # Color palette for the 3 overlays
    # overlay_colors = ["#2b83ba", "#d7191c", "#1a9641"]  # blue, red, green

    groups = sorted(df1["g"].unique())  # ['A', 'B', 'C', ..., 'O']


    def solid_palette(color_hex, n=NUM_GROUPS):
       
        rgb = np.array(sns.color_palette([color_hex])[0])
        pal = [tuple(rgb) for _ in range(n)]
        return pal

    blue_pal = solid_palette("#0044dd", NUM_GROUPS)   
    red_pal = solid_palette("#dd0000", NUM_GROUPS)   
    green_pal = solid_palette("#00dd00", NUM_GROUPS)  


    pal = solid_palette("#100000", NUM_GROUPS)
    g = sns.FacetGrid(df1, row="g", hue="g", aspect=NUM_GROUPS*0.85, height=1.4, palette=pal)


    min_len = min(
        df1.groupby("g").size().min(),
        df2.groupby("g").size().min(),
        df3.groupby("g").size().min()
    )

    # Custom plotting function: draw KDE from all three DataFrames
    def plot_all(data, color, label, **kwargs):
        group = data["g"].iloc[0]  # current group (e.g. "A")
        ax = plt.gca()
        idx = groups.index(group)
        colors = [red_pal[idx], blue_pal[idx], green_pal[idx]]
        
        for dfx, c in zip([df1, df2, df3], colors):
            subset = dfx[dfx["g"] == group]
            sns.kdeplot(
                np.random.choice(subset["x"], size=min_len, replace=False),
                bw_adjust=0.02,
                fill=True,
                alpha=0.6,
                linewidth=1.5,
                color=c,
                ax=ax,
                clip=(0,1)
            )
        #   sns.kdeplot(
        #       subset["x"],
        #       bw_adjust=0.5,
        #       fill=False,
        #       lw=2,
        #       color="white",
        #       ax=ax
        #   )

    # Map the custom function
    g.map_dataframe(plot_all)

    # Add zero baseline
    g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)

    # Label function
    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .6, label, color=color,
                ha="left", va="center", transform=ax.transAxes, fontsize=22)

    g.map(label, "x")

    # Adjust layout
    g.figure.subplots_adjust(hspace=-0.005) # g.figure.subplots_adjust(hspace=-0.25)
    g.set_titles("")
    g.set(yticks=[], ylabel="")
    g.despine(bottom=True, left=True)


    plt.xlim(0.995, 1)
    plt.xlabel("Confidence", fontsize=22)
    g.figure.text(x=0.1, y=0.5, s="Density", va="center", rotation="vertical", fontsize=22)

    plt.legend(
        loc="upper center",
        bbox_to_anchor=(0.5, 4.3),
        ncol=3,
        frameon=False,
        fontsize=22,
        labels=[ "Rtn", "Fgt", "Test"],
        handletextpad=0.3,
        columnspacing=0.7
    )
    plt.xticks([0.995, 0.9975, 1.0], fontsize=22)
    # plt.tight_layout()
    plt.savefig(f"assets/figures/conf_diff/summary_confidence_{title}_kde.pdf", bbox_inches='tight')
    plt.show()
    plt.close()


### appendix

In [ ]:
for arch, data in zip(architecture, dataset):
    for us in range(3):
        min_len = min(len(v) for v in seed_dict[0]['orig_confidence'].values())
        clip_range = (0,1)

        fig, axs = plt.subplots(1, 2, figsize=(8, 3))  

        for i in range(1): # 3
            for j, phase in enumerate(['orig_confidence', 'retrain_confidence']):  # 0: original, 1: retrain
                ax = axs[j] # i
                for label, color in zip(["retain", "forget", "test"], ["red", "blue", "green"]):
                    sns.kdeplot(
                        np.random.choice(seed_dict[i][phase][label].numpy(), size=min_len, replace=False),
                        label=label.capitalize(),
                        fill=True, linewidth=2, clip=clip_range,
                        color=color, alpha=0.4, ax=ax
                    )

                ax.set_title(f'{"Original" if phase == "orig_confidence" else "Retrain"} Model')  #  - Init Seed {i}
                ax.set_xlabel('Confidence')
                ax.set_ylabel('Density')
                ax.set_xlim(0.7, 1 + 1e-6)
                ax.grid(True)

       
        handles, labels = axs[0].get_legend_handles_labels() # axs[0, 0].get_legend_handles_labels()
        legend = fig.legend(handles, labels, loc='lower center', ncol=3, fontsize='medium')

        legend.get_frame().set_facecolor('none') 
        legend.get_frame().set_edgecolor('none') 
        plt.tight_layout(rect=[0, 0.05, 1, 1])  
        plt.show()
        fig.savefig(f"assets/figures/conf_diff/conf_diff_figures/{data}_{arch}_unlearn_seed_{us}_confidence_distribution.pdf", dpi=300, bbox_inches='tight', format='pdf')
        plt.close(fig)